In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
import numpy as np
from scipy.optimize import minimize

import matplotlib.pyplot as plt

def apx_func(x, k, theta, nu, sigma):
    n = len(x)
    res = np.zeros((n))
    for i in range(0, k):
        res += theta[i]*np.exp(-1*((x-nu[i])/sigma[i])**2)
    return res

In [ ]:
from math import cos, sin

def f(x: float) -> float:
    return 1.0 + 1.5 * x + 2.0 * cos(3.0 * x) + 3.0 * sin(7.0 * x)

In [ ]:
def min_func_hess(x, *args):
    k = args[0]
    x_e = args[1]
    y_e = args[2]
    theta = x[0:k]
    nu = x[k:2*k]
    sigma =x[2*k:3*k]
    
    y = apx_func(x_e, k, theta, nu, sigma)
    delta = y_e - y
    n = len(delta)
    
    sech2_delta = (np.cosh(delta))**(-2)
    th_delta = np.tanh(delta)
    

    res = np.zeros((3*k, 3*k))
    shifts = np.zeros((n, k))
    shifts2 = np.zeros((n, k))
    exps = np.zeros((n, k))
    
    for i in range(0, k):
        shifts[:,i] = x_e-nu[i]
        shifts2[:,i] = shifts[:,i]**2
        exps[:, i] = np.exp(-1*(shifts[:,i]/sigma[i])**2)
        
        
    # (d^2F)/(dTheta_i*dTheta_j)
    for i in range(0, k):
        for j in range(i, k):
            res[i, j] = np.sum(sech2_delta*exps[:,i]*exps[:,j])
    
    for i in range(0, k):
        for j in range(0, k):
            # (d^2F)/(dTheta_i*dNu_j)
            res[i, k+j] = -2*(theta[j]/sigma[j]**2)*np.sum(sech2_delta*exps[:,j]*shifts[:,j]*exps[:,i])
            if i == j:
                res[i, k+j] += 2*np.sum(th_delta*exps[:,j]*shifts[:,j])/sigma[j]**2
            res[i, k+j] = -res[i, k+j]
            
             # (d^2F)/(dTheta_i*dSigma_j)
            res[i, 2*k+j] = -2*(theta[j]/sigma[j]**3)*np.sum(sech2_delta*exps[:,j]*shifts2[:,j]*exps[:,i])
            if i == j:
                res[i, 2*k+j] += 2*np.sum(th_delta*shifts2[:,j]*exps[:,j])/sigma[j]**3
            res[i, 2*k+j] = -res[i, 2*k+j]
    
    
    for i in range(0, k):
        for j in range(i, k):
            #(d^2F)/(dNu_i*dNu_j)
            res[k+i, k+j] = -2*(theta[j]/sigma[j]**2)*np.sum(sech2_delta*exps[:,j]*shifts[:,j]*exps[:,i]*shifts[:,i])
            if i == j:
                res[k+i, k+j] += np.sum(th_delta*(2*shifts2[:,j]*exps[:,j]/(sigma[j]**2)-exps[:,j]))
            
            res[k+i, k+j] *= -2*theta[i]/sigma[i]**2
            
    for i in range(0, k):
        for j in range(0, k):
            #(d^2F)/(dNu_i*dSigma_j)
            res[k+i, 2*k+j] = -2*(theta[j]/sigma[j]**3)*np.sum(sech2_delta*exps[:,j]*shifts2[:,j]*exps[:,i]*shifts[:,i])
            if i == j:
                res[k+i, 2*k+j] += 2*np.sum(th_delta*exps[:,j]*shifts[:,j]**3)/sigma[j]**3
            res[k+i, 2*k+j] *= -2*theta[i]/sigma[i]**2
            
    for i in range(0, k):
        for j in range(i, k):
            #(d^2F)/(dSigma_i*dSigma_j)
            res[2*k+i, 2*k+j] = -2*(theta[j]/sigma[j]**3)*np.sum(sech2_delta*exps[:,j]*shifts2[:,j]*exps[:,i]*shifts2[:,i])
            if i == j:
                res[2*k+i, 2*k+j] += np.sum(th_delta*exps[:, j]*shifts2[:,j]**2)/(sigma[j]**3)
            
            res[2*k+i, 2*k+j] *= -2*theta[i]/sigma[i]**3 
    
    for i in range(0, 3*k):
        for j in range(i, 3*k):
            res[j, i] = res[i, j]
    return res

def min_func_jac(x, *args):
    k = args[0]
    x_e = args[1]
    y_e = args[2]
    theta = x[0:k]
    nu = x[k:2*k]
    sigma =x[2*k:3*k]
    
    y = apx_func(x_e, k, theta, nu, sigma)
    delta = y_e - y
    th_delta = np.tanh(delta)
    

    res = np.zeros((3*k))
    for i in range(0, k):
        shift = x_e - nu[i]
        exp_cur = np.exp(-1*(shift/sigma[i])**2)
        temp = th_delta*exp_cur
        
        res[i] = -np.sum(temp)
        res[k+i] = -2*theta[i]*np.sum(temp*shift)/(sigma[i]**2)
        res[2*k+i] = -2*theta[i]*np.sum(temp*shift**2)/(sigma[i]**3)
    return res

def min_func(params, *args):
    k = args[0]
    x_e = args[1]
    y_e = args[2]
    
    theta, mu, sigma = np.split(params, 3)
    assert len(theta) == len(mu) == len(sigma)  
    y = apx_func(x_e, k, theta, mu, sigma)
    delta = np.sum(np.log(np.cosh(y_e - y)))
    return delta

In [ ]:
start, stop, n = 0.0, 4.0, 100
x = np.linspace(start, stop, n)
f_vectorize = np.vectorize(f)
y_orig = f_vectorize(x)
y = y_orig + 1.25 * np.random.randn(n)

k = 8

params = np.zeros(3 * k)
step = (stop - start) / k
for i in range(k):
    params[i] = np.sum(x[i*k:k*(i+1)]) / k
    params[k+i] = start + (i + 0.5) * step
params[2*k:3*k] = step / 2
params[1] *=-1

res = minimize(fun=min_func, x0=params, args=(k, x, y), method='Nelder-Mead', tol=1e-6)

print('nit = %d, err = %f'%(res.nit, res.fun))
new_theta = res.x[0:k]
new_nu = res.x[k:2*k]
new_sigma = res.x[2*k:3*k]

new_y = apx_func(x, k, new_theta, new_nu, new_sigma)
err_rel = sum(abs((new_y-y_orig)/y_orig))/n
print('rel_err = ', err_rel)

plt.plot(x, new_y, 'red')
plt.plot(x, y, '*')
plt.plot(x, y_orig)

In [ ]:
res = minimize(fun=min_func, x0=params, args=(k, x, y), method='BFGS', jac=min_func_jac, tol=1e-6)

print('nit = %d, err = %f'%(res.nit, res.fun))
new_theta = res.x[0:k]
new_nu = res.x[k:2*k]
new_sigma = res.x[2*k:3*k]

new_y = apx_func(x, k, new_theta, new_nu, new_sigma)
err_rel = sum(abs((new_y-y_orig)/y_orig))/n
print('rel_err = ', err_rel)

plt.plot(x, new_y, 'red')
plt.plot(x, y, '*')
plt.plot(x, y_orig)

In [ ]:
res = minimize(fun=min_func, x0=params, args=(k, x, y), method='Newton-CG', hess=min_func_hess, jac=min_func_jac)

print('nit = %d, err = %f'%(res.nit, res.fun))
new_theta = res.x[0:k]
new_nu = res.x[k:2*k]
new_sigma = res.x[2*k:3*k]

new_y = apx_func(x, k, new_theta, new_nu, new_sigma)
err_rel = sum(abs((new_y-y_orig)/y_orig))/n
print('rel_err = ', err_rel)

plt.plot(x, new_y, 'red')
plt.plot(x, y, '*')
plt.plot(x, y_orig)